In [1]:
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile
from ahocorasick import Automaton
from tqdm.auto import tqdm

# Code to clean the data

In [2]:
FILE = "./italian_restaurants.zip"
zip_file = ZipFile(FILE)
csv_file = zip_file.namelist()[0]
data = pd.read_csv(zip_file.open(csv_file))

/var/folders/jb/ntxm50wx2wv90pnq36yd8w940000gn/T/ipykernel_60618/804822752.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(zip_file.open(csv_file))


In [3]:
data.iloc[0]

Unnamed: 0                                                                      650043
restaurant_link                                                    g10029467-d18338571
restaurant_name                                                                L'Ortal
original_location                    ["Europe", "Italy", "Friuli Venezia Giulia", "...
country                                                                          Italy
region                                                           Friuli Venezia Giulia
province                                                         Province of Pordenone
city                                                                               NaN
address                                  Via Jacopo Ortis 15, 33090, Vito d'Asio Italy
latitude                                                                      46.22801
longitude                                                                      12.9397
claimed                                    

Removing useless fields

In [4]:
df = data.drop(["claimed", 
           "awards", 
           "popularity_detailed", 
           "popularity_generic", 
           "price_range", 
           "features", 
           "open_days_per_week", 
           "open_hours_per_week", 
           "working_shifts_per_week",
           "default_language",
           "reviews_count_in_default_language",
           "excellent",
           "very_good",
           "average",
           "poor",
           "terrible",
           "food",
           "service",
           "value",
           "atmosphere",
           "keywords"], axis=1)

Populating the "City" field taking it from the address

In [5]:
comuni_df = pd.read_csv("./elenco_comuni_italiani.csv", encoding='iso-8859-1', delimiter=";")

In [6]:
#TODO: "Rome" non viene preso, quindi risolvi questa cosa
def build_automaton(keywords):
    automaton = Automaton()
    for keyword in keywords:
        automaton.add_word(keyword, keyword)
    automaton.make_automaton()
    return automaton

keywords = [row['Denominazione in italiano'].lower() for _, row in comuni_df.iterrows()]
automaton = build_automaton(keywords)

def find_comune(address):
    found = set()
    for item in automaton.iter(",".join(address.split(",")[1:]).lower()):
        found.add(item[1])
    if found:
        return max(found, key=len)
    else:
        return None

tqdm.pandas()
df['city'] = df['address'].progress_apply(find_comune)



  0%|          | 0/224763 [00:00<?, ?it/s]

In [7]:
final_df = df.copy()
final_df['total_reviews_count'] = final_df['total_reviews_count'].fillna(0.0)
final_df['avg_rating'] = final_df['avg_rating'].fillna(0.0)
final_df = final_df.fillna("")
final_df = final_df[final_df["latitude"] != ""]
final_df = final_df[final_df["longitude"] != ""]

In [11]:
df = final_df.reset_index()

,index,Unnamed: 0,restaurant_link,restaurant_name,original_location,country,region,province,city,address,...,price_level,meals,cuisines,special_diets,vegetarian_friendly,vegan_options,gluten_free,original_open_hours,avg_rating,total_reviews_count
0,0,650043,g10029467-d18338571,L'Ortal,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,vito d'asio,"Via Jacopo Ortis 15, 33090, Vito d'Asio Italy",...,€,,"Italian, Northern-Italian",,N,N,N,,5.0,5.0
1,1,650044,g10029467-d23200955,Osteria con Cucina Lorenzini,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,vito d'asio,"Via Chiamp, 18, 33090, Vito d'Asio Italy",...,,"Breakfast, Lunch, Dinner",,,N,N,N,"{""Mon"": [""08:00-19:00""], ""Tue"": [""08:00-19:00""...",0.0,0.0
2,2,650045,g10029597-d6878487,Ristorante Da Renzo,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,vito d'asio,"Via Galantz 2, 33090 San Francesco, Vito d'Asi...",...,€,"Lunch, Dinner, Breakfast",Italian,Vegetarian Friendly,Y,N,N,"{""Mon"": [""07:00-22:00""], ""Tue"": [""07:00-22:00""...",4.5,65.0
3,3,650046,g10038815-d14177798,Pizza Point,"[""Europe"", ""Italy"", ""Trentino-Alto Adige"", ""Pr...",Italy,Trentino-Alto Adige,Province of South Tyrol,valdaora,"Via del Borgo 4B, 39030 Valdaora di Sopra, Val...",...,€,,Pizza,,N,N,N,,5.0,4.0
4,4,650047,g10045458-d12940567,Pizzeria Snoopy 2,"[""Europe"", ""Italy"", ""Emilia-Romagna"", ""Provinc...",Italy,Emilia-Romagna,Province of Bologna,dozza,"Piazza Liberta' 14, 40060 Toscanella, Dozza Italy",...,€,,"Italian, Pizza",,N,N,N,,3.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222598,224758,874801,g9885748-d12879403,La Locanda dell'Eremita,"[""Europe"", ""Italy"", ""Campania"", ""Province of C...",Italy,Campania,Province of Caserta,caserta,"Via San Vitaliano 30, 81100 Casola, Caserta Italy",...,,,Italian,,N,N,N,,4.0,7.0
222599,224759,874802,g9885748-d4002177,Ristorante Pizzeria La Fenice,"[""Europe"", ""Italy"", ""Campania"", ""Province of C...",Italy,Campania,Province of Caserta,caserta,"Via Dottore Ferdinando Rossi 41, 81100 Casola,...",...,€,"Lunch, Dinner, After-hours","Italian, Pizza, Mediterranean, Neapolitan, Cam...",,N,N,N,"{""Mon"": [""17:30-23:45""], ""Tue"": [""18:30-23:00""...",4.0,190.0
222600,224760,874803,g9885748-d6905025,Agriturismo l'Eremo,"[""Europe"", ""Italy"", ""Campania"", ""Province of C...",Italy,Campania,Province of Caserta,caserta,"Via San Vitaliano 34, 81100 Casola, Caserta Italy",...,€€-€€€,"Lunch, Dinner","Italian, Mediterranean",,N,N,N,"{""Mon"": [""11:00-00:00""], ""Tue"": [""11:00-00:00""...",4.0,117.0
222601,224761,874804,g9885970-d15265965,L'Etoile,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,aviano,"Viale San Giorgio 9, 33081 Castello d'Aviano, ...",...,,,,,N,N,N,,4.5,3.0


### Save the dataframe as a zipped file

In [13]:
from zipfile import ZipFile
import os

DF_PATH = "./italian_restaurant_clean.csv"
ZIP_PATH = "./italian_restaurant_clean.zip"
final_df.to_csv(DF_PATH)
with ZipFile(ZIP_PATH, "w") as myzip:
    myzip.write(DF_PATH)

os.remove(DF_PATH)

# Code to insert the restaurants in the database 

In [ ]:
df = pd.read_csv("./italian_restaurant_clean.zip")

In [16]:
URL = 'http://localhost:3000/restaurants/create-restaurant'

def assign_image(index):
    TOTAL_NUMBER_OF_IMAGES = 4761
    return index % TOTAL_NUMBER_OF_IMAGES

def create_restaurant(row):
    body = {
            "name": row["restaurant_name"],
            "imageName": f"restaurant_image_{assign_image(row['index'])}",
            "address": row["address"],
            "latitude": float(row["latitude"]),
            "longitude": float(row["longitude"]),
            "country": row["country"],
            "region": row["region"],
            "province": row["province"],
            "city": row["city"],
            "tags": row["top_tags"],
            "cuisines": row["cuisines"],
            "specialDiets": row["special_diets"],
            "priceLevel": row["price_level"],
            "meals": row["meals"],
            "avgRating": float(row["avg_rating"]),
            "vegetarianFriendly": row["vegetarian_friendly"] == "Y",
            "veganFriendly": row["vegan_options"] == "Y",
            "glutenFree": row["gluten_free"] == "Y",
            "reviewsNumber": int(row["total_reviews_count"])
        }

    
    result = requests.post(URL, json=body)

tqdm.pandas()
df.progress_apply(create_restaurant, axis=1)

  0%|          | 0/222603 [00:00<?, ?it/s]

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))